In [83]:
# import pandas as pd
# df = pd.read_csv("file:///opt/spark/notebooks/stop_timesold.txt")
# df.to_csv("yo")


In [6]:
import kudu
from kudu.client import Partitioning
from datetime import datetime

# Connect to Kudu master server
client = kudu.connect(host='172.18.0.2', port=7051) # in our case kudu-master if you havent changed anything

# Define a schema for a new table
builder = kudu.schema_builder()
builder.add_column('key').type(kudu.int64).nullable(False).primary_key()
builder.add_column('ts_val', type_=kudu.unixtime_micros, nullable=False, compression='lz4')
schema = builder.build()

# Define partitioning schema
partitioning = Partitioning().add_hash_partitions(column_names=['key'], num_buckets=3)

# Create new table
# client.create_table('python-example2', schema, partitioning)

# Open a table
table = client.table('python-example2')

# Create a new session so that we can apply write operations
session = client.new_session()

# Insert a row
op = table.new_insert({'key': 5, 'ts_val': datetime.utcnow()})
print("t1")
print(table.show())
session.apply(op)

# Upsert a row
op = table.new_upsert({'key': 6, 'ts_val': "2016-01-01T00:00:00.000000"})
session.apply(op)

# Updating a row
op = table.new_update({'key': 5, 'ts_val': ("2017-01-01", "%Y-%m-%d")})
session.apply(op)

# Delete a row
op = table.new_delete({'key': 5})
session.apply(op)
# Flush write operations, if failures occur, capture print them.
try:
   session.flush()
except kudu.KuduBadStatus as e:
   print(session.get_pending_errors())

# Create a scanner and add a predicate
scanner = table.scanner()
scanner.add_predicate(table['ts_val'] == datetime(2017, 1, 1))

# Open Scanner and read all tuples
# Note: This doesn't scale for large scans
result = scanner.open().read_all_tuples()
print(result)

t1


AttributeError: 'kudu.client.Table' object has no attribute 'show'

In [20]:
session = client.new_session()

In [63]:
builder = kudu.schema_builder()
builder.add_column('route_id').type(kudu.string).nullable(False).primary_key()
builder.add_column('route_short_name',type_=kudu.string,nullable = True, compression='lz4')
builder.add_column('route_long_name', type_=kudu.string, nullable=True, compression='lz4')
builder.add_column('route_type', type_=kudu.string, nullable=True, compression='lz4')
builder.add_column('agency_id', type_=kudu.string, nullable=False, compression='lz4')


schema = builder.build()
partitioning = Partitioning().add_hash_partitions(column_names=['route_id'], num_buckets=30)
# client.create_table('kudu_routes', schema, partitioning)
table = client.table('kudu_routes');

df1 = spark.read.option("delimiter",",").option("header",True).csv("hdfs://namenode:9000//routes.txt")
# df1 = spark.read.option("delimiter",",").option("header",True).csv("file:///opt/spark/notebooks/routes.txt")
df1 = df1.select(df1.route_id, df1.route_short_name, df1.route_long_name, df1.route_type,df1.agency_id)
n_rows = df1.count()
obj = df1.collect()
for i in range(n_rows):
    if(i<3):
        print(obj[i])
    op = table.new_insert(obj[i])
    session.apply(op)
try:
   session.flush()
except kudu.KuduBadStatus as e:
   print(session.get_pending_errors())

routes = spark.read.format('org.apache.kudu.spark.kudu').option('kudu.master',"172.18.0.2:7051").option('kudu.table',"kudu_routes").load()
# kuduDF.createOrReplaceTempView("mock_routes")



# Open a table




Row(route_id='0', route_short_name=None, route_long_name='193STLDOWN', route_type='3', agency_id='DIMTS')
Row(route_id='1', route_short_name=None, route_long_name='125EXTDOWN', route_type='3', agency_id='DIMTS')
Row(route_id='2', route_short_name=None, route_long_name='827LNKSTLUP', route_type='3', agency_id='DIMTS')
([KuduError('b'Already present: key already present''), KuduError('b'Already present: key already present''), KuduError('b'Already present: key already present''), KuduError('b'Already present: key already present''), KuduError('b'Already present: key already present''), KuduError('b'Already present: key already present''), KuduError('b'Already present: key already present''), KuduError('b'Already present: key already present''), KuduError('b'Already present: key already present''), KuduError('b'Already present: key already present''), KuduError('b'Already present: key already present''), KuduError('b'Already present: key already present''), KuduError('b'Already present: k

In [56]:
pip install pandas

     |████████████████████████████████| 11.3 MB 30 kB/s  eta 0:00:01    |█████████████▌                  | 4.8 MB 7.0 MB/s eta 0:00:01
     |████████████████████████████████| 15.7 MB 93 kB/s s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [64]:

routes.show()

+--------+----------------+---------------+----------+---------+
|route_id|route_short_name|route_long_name|route_type|agency_id|
+--------+----------------+---------------+----------+---------+
|     106|            null|       818BDOWN|         3|    DIMTS|
|    1084|            null|       410ACLUP|         3|    DIMTS|
|    1125|            null|     128STLDOWN|         3|    DIMTS|
|    1126|            null|       539BDOWN|         3|    DIMTS|
|    1166|            null|       149ADOWN|         3|    DIMTS|
|    1193|            null|        834DOWN|         3|    DIMTS|
|    1195|            null|        405DOWN|         3|    DIMTS|
|    1196|            null|   418ALNKSTLUP|         3|    DIMTS|
|     120|            null|       720ADOWN|         3|    DIMTS|
|    1212|            null|         979AUP|         3|    DIMTS|
|    1248|            null|       448STLUP|         3|    DIMTS|
|    1252|            null|        907DOWN|         3|    DIMTS|
|     131|            nul

In [46]:
builder = kudu.schema_builder()
builder.add_column('trip_id').type(kudu.string).nullable(False).primary_key()
builder.add_column('route_id',type_=kudu.string,nullable = True, compression='lz4')
builder.add_column('service_id', type_=kudu.string, nullable=True, compression='lz4')
builder.add_column('shape_id', type_=kudu.string, nullable=True, compression='lz4')


schema = builder.build()
partitioning = Partitioning().add_hash_partitions(column_names=['trip_id'], num_buckets=10)
# client.create_table('kudu_trips', schema, partitioning)
table = client.table('kudu_trips');

df1 = spark.read.option("delimiter",",").option("header",True).csv("hdfs://namenode:9000//trips.txt")
# df1 = spark.read.option("delimiter",",").option("header",True).csv("file:///opt/spark/notebooks/trips.txt")

df1 = df1.select(df1.trip_id, df1.route_id, df1.service_id, df1.shape_id)
n_rows = df1.count()
obj = df1.collect()
for i in range(n_rows):
    if(i<3):
        print(obj[i])
    op = table.new_insert(obj[i])
    session.apply(op)
try:
   session.flush()
except kudu.KuduBadStatus as e:
   print(session.get_pending_errors())

trips = spark.read.format('org.apache.kudu.spark.kudu').option('kudu.master',"172.18.0.2:7051").option('kudu.table',"kudu_trips").load()
# kuduDF.createOrReplaceTempView("mock_trips")

Row(trip_id='0_20_20', route_id='0', service_id='1', shape_id=None)
Row(trip_id='0_20_30', route_id='0', service_id='1', shape_id=None)
Row(trip_id='0_20_40', route_id='0', service_id='1', shape_id=None)
([KuduError('b'Already present: key already present''), KuduError('b'Already present: key already present''), KuduError('b'Already present: key already present''), KuduError('b'Already present: key already present''), KuduError('b'Already present: key already present''), KuduError('b'Already present: key already present''), KuduError('b'Already present: key already present''), KuduError('b'Already present: key already present''), KuduError('b'Already present: key already present''), KuduError('b'Already present: key already present''), KuduError('b'Already present: key already present''), KuduError('b'Already present: key already present''), KuduError('b'Already present: key already present''), KuduError('b'Already present: key already present''), KuduError('b'Already present: key alre

In [111]:
# trips.show()

In [74]:
stops.count()

4192

In [48]:
builder = kudu.schema_builder()
builder.add_column('stop_id').type(kudu.int64).nullable(False).primary_key()
builder.add_column('stop_code',type_=kudu.string,nullable = True, compression='lz4')
builder.add_column('stop_name', type_=kudu.string, nullable=True, compression='lz4')
builder.add_column('stop_lat', type_=kudu.float, nullable=True, compression='lz4')
builder.add_column('stop_lon', type_=kudu.float, nullable=True, compression='lz4')
builder.add_column('zone_id', type_=kudu.int64, nullable=True, compression='lz4')



schema = builder.build()
partitioning = Partitioning().add_hash_partitions(column_names=['stop_id'], num_buckets=10)
# client.create_table('kudu_stops', schema, partitioning)
table = client.table('kudu_stops');

df1 = spark.read.option("delimiter",",").option("header",True).csv("hdfs://namenode:9000//stops.txt")
# df1 = spark.read.option("delimiter",",").option("header",True).csv("file:///opt/spark/notebooks/stops.txt")

df1 = df1 \
  .withColumn("stop_id" ,
              df1["stop_id"]
              .cast('int'))   \
  .withColumn("stop_lat",
              df1["stop_lat"]
              .cast('float'))    \
  .withColumn("stop_lon"  ,
              df1["stop_lon"]
              .cast('float')) \
    .withColumn("zone_id"  ,
              df1["zone_id"]
              .cast('int')) \

#df1 = df1.select(df1.trip_id, df1.route_id, df1.service_id, df1.shape_id)
n_rows = df1.count()
obj = df1.collect()
for i in range(n_rows):
    if(i<3):
        print(obj[i])
    op = table.new_insert(obj[i])
    session.apply(op)
try:
   session.flush()
except kudu.KuduBadStatus as e:
   print(session.get_pending_errors())

stops = spark.read.format('org.apache.kudu.spark.kudu').option('kudu.master',"172.18.0.2:7051").option('kudu.table',"kudu_stops").load()
# kuduDF.createOrReplaceTempView("mock_stops")

Row(stop_id=0, stop_code='DC4539', stop_name='Narela Terminal', stop_lat=28.85200309753418, stop_lon=77.08805847167969, zone_id=0)
Row(stop_id=1, stop_code='DC4540', stop_name='Police Station Narela', stop_lat=28.85336685180664, stop_lon=77.08853149414062, zone_id=1)
Row(stop_id=2, stop_code='DC4541', stop_name='Safiyabad Crossing', stop_lat=28.855266571044922, stop_lon=77.08972930908203, zone_id=2)
([KuduError('b'Already present: key already present''), KuduError('b'Already present: key already present''), KuduError('b'Already present: key already present''), KuduError('b'Already present: key already present''), KuduError('b'Already present: key already present''), KuduError('b'Already present: key already present''), KuduError('b'Already present: key already present''), KuduError('b'Already present: key already present''), KuduError('b'Already present: key already present''), KuduError('b'Already present: key already present''), KuduError('b'Already present: key already present''), K

In [49]:

stops.show()

2022-03-04 15:49:40 WARN  NetUtil:106 - Slow DNS lookup! Resolved IP of `377039d9f470' to 377039d9f470/172.18.0.4 in 4949464ns
2022-03-04 15:49:40 WARN  NetUtil:106 - Slow DNS lookup! Resolved IP of `e8eb96a13b12' to e8eb96a13b12/172.18.0.5 in 5028644ns
+-------+---------+--------------------+---------+---------+-------+
|stop_id|stop_code|           stop_name| stop_lat| stop_lon|zone_id|
+-------+---------+--------------------+---------+---------+-------+
|      8|   DC4547|A-7 Narela Sec- 1...|28.849049| 77.09845|      8|
|     13|   DC4552|         Kurani More|28.840836| 77.09053|     13|
|     21|   DC4560|JJ Colony Bawana ...|28.806835| 77.04801|     21|
|     33|   U01867|             Azadpur|  28.7063|  77.1795|     33|
|     34|   U01929|           Bara Bagh|  28.7017|  77.1831|     34|
|     54|   U03126|        Shastri Park|  28.6726|  77.2571|     54|
|     76|   U00143|Tikri Border Metr...|28.688171| 76.96446|     76|
|     83|   U00290|      Ghevra Village|28.696285| 76.99

In [114]:
# client.delete_table("kudu_stop_times")

In [108]:
builder = kudu.schema_builder()
builder.add_column('key').type(kudu.string).nullable(False).primary_key()
# builder.add_column('stop_sequence').type(kudu.string).nullable(False)
# builder.add_column('stop_id').type(kudu.string).nullable(False).primary_key()
# builder.add_column('trip_id').type(kudu.string).nullable(False).primary_key()
builder.add_column('trip_id', type_=kudu.string, nullable=True, compression='lz4')



builder.add_column('arrival_time', type_=kudu.string, nullable=True, compression='lz4')
builder.add_column('departure_time', type_=kudu.string, nullable=True, compression='lz4')
builder.add_column('stop_id',type_=kudu.string,nullable = True, compression='lz4')
builder.add_column('stop_sequence',type_=kudu.string,nullable = True, compression='lz4')

schema = builder.build()
partitioning = Partitioning().add_hash_partitions(column_names=['key'], num_buckets=10)
client.create_table('kudu_stop_times', schema, partitioning)
table = client.table('kudu_stop_times');

df1 = spark.read.option("delimiter",",").option("header",True).csv("hdfs://namenode:9000//stop_times.txt")
# df1 = spark.read.option("delimiter",",").option("header",True).csv("file:///opt/spark/notebooks/stop_times.txt")
# df1 = df1.select(, df1.trip_id,  df1.arrival_time,df1.departure_time, df1.stop_id, df1.stop_sequence)
df1.write.format('org.apache.kudu.spark.kudu').option('kudu.master',"172.18.0.2:7051").option('kudu.table',"kudu_stop_times").mode('append').save()
print(23)
try:
   session.flush()
except kudu.KuduBadStatus as e:
   print(session.get_pending_errors())
print(33)
stop_times = spark.read.format('org.apache.kudu.spark.kudu').option('kudu.master',"172.18.0.2:7051").option('kudu.table',"kudu_stop_times").load()
# stop_times.createOrReplaceTempView("mock_stop_times")
print(43)

23
33
43


In [109]:
stop_times.count()

2250290

In [110]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType
from pyspark.sql.functions import to_timestamp
from pyspark.sql.types import IntegerType
# print(stop_times.show(5))
stop_times = stop_times.withColumn("stop_sequence", stop_times["stop_sequence"].cast(IntegerType()))
stop_times = stop_times.withColumn("arrival_time", to_timestamp(stop_times["arrival_time"]))
stop_times = stop_times.withColumn("departure_time", to_timestamp(stop_times["departure_time"]))

 


# print(stop_times.show(5))
#routes.show(5)
#stops.show(5)
#trips.show(5)

 
print("1.")
rt = routes.join(trips, on=['route_id'])
rt = rt.dropDuplicates(["route_id","trip_id"])
rt.groupBy("route_id").count().orderBy('route_id').show(rt.count())
rts = rt.join(stop_times, on=['trip_id'])
rts = rts.dropDuplicates(["route_id","stop_id"])

rts.groupBy("route_id").count().orderBy('route_id').show(rts.count())


print("2.")
print("a)")
v1 = (stop_times.join(routes.join(trips, on='route_id'), on='trip_id')) .select('route_id',  'stop_id', 'stop_sequence')
v2 = v1.groupBy('route_id').min('stop_sequence')
v2 = v2.withColumnRenamed("min(stop_sequence)", "stop_sequence")
v3 = (v1.join(v2, on=['route_id','stop_sequence'])).select('route_id', 'stop_id')
v3 = v3.dropDuplicates(["route_id","stop_id"])
v4 = (v3.join(stops, on=['stop_id'])).select('route_id', 'stop_name', 'stop_lat', 'stop_lon')
v4.show(v4.count(), truncate = False)


print("b)")
v1 = (stop_times.join(routes.join(trips, on='route_id'), on='trip_id')) .select('route_id',  'stop_id', 'stop_sequence')
v2 = v1.groupBy('route_id').max('stop_sequence')
v2 = v2.withColumnRenamed("max(stop_sequence)", "stop_sequence")
v3 = (v1.join(v2, on=['route_id','stop_sequence'])).select('route_id', 'stop_id')
v3 = v3.dropDuplicates(["route_id","stop_id"])
v4 = (v3.join(stops, on=['stop_id'])).select('route_id', 'stop_name', 'stop_lat', 'stop_lon')
v4.show(v4.count(), truncate = False)




print("3.")
v1 = stop_times.filter(stop_times.stop_id == '469').select('trip_id')
# v1.show() 
v2 = (v1.join(trips, on=['trip_id'])).select('route_id').distinct()
# v2.show()
v3 = (v2.join(trips, on=['route_id'])).select('route_id', 'trip_id')
# v3.show()
v4 = (v3.join(stop_times, on = ['trip_id'])).select('route_id', 'stop_sequence')
# v4.show()
v5 = v4.groupBy('route_id').max('stop_sequence')
v5 = v5.withColumnRenamed("max(stop_sequence)", "maxdistance")
# v5.show()
minseq = v5.agg({'maxdistance': 'min'})
minseq = minseq.withColumnRenamed("min(maxdistance)", "shortestroute")
minvalue = minseq.collect()[0][0]
# print("minvalue"+str(minvalue))

v6 = v5.filter( v5.maxdistance == minvalue).select('route_id')
# v6.show()
v7 = (trips.join(v6, on = ['route_id'])).select('trip_id') 
# v7.show()
v8 =  (stop_times.join(v7, on = ['trip_id'] )).select('trip_id', 'arrival_time')
v8.show()




print("4.")
v1 = stop_times.groupBy("trip_id").agg({"arrival_time":'min', "departure_time" : 'max'})
print(v1.dtypes)
v1.show( )
v2 = v1.withColumnRenamed("min(arrival_time)", "mintime")
v2 = v2.withColumnRenamed("max(departure_time)", "maxtime")
v2 = v2.withColumn("mintimesec", v2["mintime"].cast("long"))
v2 = v2.withColumn("maxtimesec", v2["maxtime"].cast("long"))
# v2.show( )
v3 = v2.withColumn("diff", v2["maxtimesec"] - v2["mintimesec"])
# v3.show()
v4 = v3.orderBy('diff', ascending = False).select('trip_id')
v4.show()
 

print("5.")
t1 = stop_times.filter(stop_times.stop_id == '469').select('trip_id')
# t1.show() 
t2 = (t1.join(trips, on=['trip_id'])).select('route_id').distinct()
# t2.show() 
t3 = (t2.join(trips, on = ['route_id'])).select('trip_id').distinct()
t4 = stop_times.join(t3, on = ['trip_id']).select('stop_id').distinct()

t5 = stops.join(t4, stops.stop_id == t4.stop_id,  "left_anti").select('stop_id').distinct()

t6 = t5.join(stops, on = ['stop_id']).select('stop_name')
t6.show(truncate = False)

 
print('6.')
print(stop_times.dtypes)
v1 = stop_times.filter(stop_times.stop_sequence == 0 ).select('stop_id')
v2 = v1.groupBy("stop_id").count()
# v2.show()
v2 = v2.orderBy('count', ascending = False)
# v2.show()
v3 = v2.limit(3)
# v3.show()
v4 =  (v3.join(stops,  on = ['stop_id'] )).select('stop_id', 'stop_name')
v4.show(v4.count(), truncate = False)
 


1.
2022-03-04 17:11:05 WARN  NetUtil:106 - Slow DNS lookup! Resolved IP of `377039d9f470' to 377039d9f470/172.18.0.4 in 329278365ns


+--------+-----+
|route_id|count|
+--------+-----+
|       0|   10|
|       1|   82|
|      10|   61|
|     100|   75|
|    1000|   11|
|    1001|   48|
|    1002|   20|
|    1003|   86|
|    1004|   12|
|    1006|   79|
|    1007|   78|
|    1008|   18|
|    1009|    7|
|     101|   54|
|    1010|   22|
|    1013|    4|
|    1014|   96|
|    1015|   13|
|    1016|   92|
|    1017|   65|
|    1018|   53|
|    1019|   72|
|     102|   41|
|    1020|   59|
|    1021|   60|
|    1022|   19|
|    1023|   36|
|    1024|   30|
|    1025|   80|
|    1026|   48|
|    1027|   68|
|     103|   71|
|    1030|   21|
|    1031|   96|
|    1032|    4|
|    1033|   13|
|    1034|   72|
|    1035|   89|
|    1036|   47|
|    1037|   79|
|    1038|   39|
|    1039|   16|
|     104|   19|
|    1040|    1|
|    1041|   64|
|    1043|   53|
|    1044|    5|
|    1045|   98|
|    1046|   82|
|    1047|   65|
|    1048|   87|
|    1049|    7|
|     105|   87|
|    1050|   31|
|    1051|   10|
|    1052|    

+--------+-----+
|route_id|count|
+--------+-----+
|       0|   20|
|       1|   46|
|      10|   33|
|     100|   51|
|    1000|   45|
|    1001|   49|
|    1002|   39|
|    1003|   28|
|    1004|   12|
|    1006|   32|
|    1007|   46|
|    1008|   89|
|    1009|   22|
|     101|   52|
|    1010|    8|
|    1013|   50|
|    1014|   58|
|    1015|   36|
|    1016|   52|
|    1017|   49|
|    1018|   71|
|    1019|   66|
|     102|   54|
|    1020|   67|
|    1021|   45|
|    1022|   72|
|    1023|   31|
|    1024|   39|
|    1025|   48|
|    1026|   50|
|    1027|   84|
|     103|   37|
|    1030|   34|
|    1031|   51|
|    1032|    9|
|    1033|   36|
|    1034|   39|
|    1035|   43|
|    1036|   51|
|    1037|   51|
|    1038|   16|
|    1039|   78|
|     104|   12|
|    1040|   49|
|    1041|   47|
|    1043|   44|
|    1044|   27|
|    1045|   60|
|    1046|   56|
|    1047|   58|
|    1048|   58|
|    1049|   43|
|     105|   56|
|    1050|   35|
|    1051|   41|
|    1052|    

+--------+---------------------------------------------------------+---------+---------+
|route_id|stop_name                                                |stop_lat |stop_lon |
+--------+---------------------------------------------------------+---------+---------+
|1186    |Ashok Vihar Water Tank                                   |28.687017|77.17367 |
|380     |Jharoda Village (Burari)                                 |28.730804|77.210365|
|1133    |Jharoda Village (Burari)                                 |28.730804|77.210365|
|310     |PS Kamla Market                                          |28.6429  |77.2267  |
|699     |Old Delhi Railway Station (Fatehpuri)                    |28.660295|77.2249  |
|865     |Old Delhi Railway Station (Fatehpuri)                    |28.660295|77.2249  |
|778     |Old Delhi Railway Station (Fatehpuri)                    |28.660295|77.2249  |
|511     |Old Delhi Railway Station (Fatehpuri)                    |28.660295|77.2249  |
|199     |Old Delhi R

+--------+---------------------------------------------------------+---------+---------+
|route_id|stop_name                                                |stop_lat |stop_lon |
+--------+---------------------------------------------------------+---------+---------+
|6       |Dhaula Kuan                                              |28.592987|77.162865|
|685     |Dhaula Kuan                                              |28.592987|77.162865|
|513     |Rohini Sec 23 Pkt 1 (Green Hill Aptt.)                   |28.719309|77.08471 |
|19      |Rohini Sec 23 Pkt 1 (Green Hill Aptt.)                   |28.719309|77.08471 |
|670     |Rohini Sec 23 Pkt 1 (Green Hill Aptt.)                   |28.719309|77.08471 |
|1058    |Old Delhi Railway Station (Fatehpuri)                    |28.660295|77.2249  |
|826     |Old Delhi Railway Station (Fatehpuri)                    |28.660295|77.2249  |
|1001    |Old Delhi Railway Station (Fatehpuri)                    |28.660295|77.2249  |
|73      |Rewla Khanp

...

2022-03-04 17:13:20 WARN  NetUtil:106 - Slow DNS lookup! Resolved IP of `e8eb96a13b12' to e8eb96a13b12/172.18.0.5 in 387149939ns


+---------+
|  trip_id|
+---------+
|261_08_00|
|261_17_40|
|261_17_20|
|261_07_40|
|261_09_10|
|261_16_50|
|261_10_40|
|261_11_50|
|261_13_40|
|261_15_10|
|261_14_40|
|261_07_50|
|261_12_20|
|261_14_10|
|261_18_00|
|261_10_10|
|261_14_00|
|261_10_00|
|261_11_30|
|261_08_40|
+---------+
only showing top 20 rows

5.


+--------------------------------------+
|stop_name                             |
+--------------------------------------+
|Mukarba Chowk                         |
|Mahindra Park                         |
|Vivekanand Marg / Minto Road          |
|Delhi Gate / Ambedkar Stadium Terminal|
|Samalkha Village                      |
|North Paint Nursing Home              |
|Aruna Asaf Ali Hospital               |
|Kewal Park                            |
|Dwarka Puri                           |
|Tyag Raj Marg                         |
|Ashok Vihar Water Tank                |
|Choona Factory                        |
|Prithviraj Road Crossing              |
|Jharoda Village (Burari)              |
|Golf Club                             |
|Radha Swami Satsang                   |
|Pitampura CD-Block                    |
|Rohini Sec 23 Pkt 1 (Green Hill Aptt.)|
|DDA Flats Lado Sarai                  |
|Kapashera More                        |
+--------------------------------------+
only showing top

[Stage 581:============================================>           (8 + 2) / 10]

+-------+-------------------------+
|stop_id|stop_name                |
+-------+-------------------------+
|770    |Anand Vihar ISBT Terminal|
|706    |Uttam Nagar Terminal     |
|111    |Old Delhi Railway Station|
+-------+-------------------------+

